In [83]:
#importing the necessary libraries

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
print('import successful')

import successful


In [84]:
#obtaining the wikipedia page and using lxml parser
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [85]:
#print(soup.prettify()) - used the prettify method to check the page formatting and have commented out once done

In [86]:
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


Using the BeautifulSoup library to parse the data from the postcode table in the webpage, as we can see the required elements are separated by 'td' tags

In [87]:
postcode_table = soup.find('table', class_= 'wikitable sortable')
table_elements = postcode_table.findAll('td')
print(table_elements[:20])

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>]


In [88]:
len(table_elements) #finding the total number of elements in the table

867

Using a for-loop to split the table elements into separate rows, where the first,second and third elements of each row corresponds to postcode, bourogh and neighbourhood respectively. All the rows are combined to form the list: data_table

In [89]:
data_table=[]
for i in range(0, 867, 3):
    split_table_in_columns = [[table_elements[i], table_elements[i+1], table_elements[i+2]] ]
    postcode = table_elements[i].text.strip()
    bourough = table_elements[i+1].text.strip()
    neighbourhood = table_elements[i+2].text.strip()
    data_table.append([postcode, bourough, neighbourhood])
data_table

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

In [90]:
type(data_table)

list

The data_table is cast into a pandas dataframe, and the columns are named

In [91]:
df_CanadaPostcodes = pd.DataFrame(data_table)
df_CanadaPostcodes.columns = ['Postcode', 'Bourough', 'Neighbourhood']
df_CanadaPostcodes.head(10)


,Postcode,Bourough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [92]:
df_CanadaPostcodes.shape #checking the dimensions of the dataframe

(289, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned, and the corresponding rows are removed from the dataframe

In [93]:
df_CanadaPostcodes = df_CanadaPostcodes[~df_CanadaPostcodes['Bourough'].isin(['Not assigned'])]
df_CanadaPostcodes.reset_index(inplace=True)
df_CanadaPostcodes.drop('index', axis=1, inplace=True)
df_CanadaPostcodes.head(10)


,Postcode,Bourough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [94]:
df_CanadaPostcodes.shape #checking the dimensions of the dataframe

(212, 3)

More than one neighborhood can exist in one postal code area as can be seen from the above dataframe. For example, in the table on the Wikipedia page, we will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma in the below dataframe. This is done by using the groupby method on Postcode and Bourough, and using the join method to combine the neighbourhoods (saparated by commas) having same values of Postcode and Bourough

In [95]:
df_CanadaPostcodes = df_CanadaPostcodes.groupby(['Postcode','Bourough'])['Neighbourhood'].apply(', '.join).reset_index()
df_CanadaPostcodes

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [96]:
missing_neighbourhoods = df_CanadaPostcodes.Neighbourhood.values == 'Not assigned'
df_CanadaPostcodes.Neighbourhood[missing_neighbourhoods] = df_CanadaPostcodes.Bourough[missing_neighbourhoods]
df_CanadaPostcodes

,Postcode,Bourough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [97]:
df_CanadaPostcodes.shape #checking the dimensions of the dataframe

(103, 3)

In [99]:
#importing the csv file that has the geographical coordinates of each postal code and casting it into a new pandas dataframe called geo_data
urlfordata="http://cocl.us/Geospatial_data" 
geo_data=pd.read_csv(urlfordata)
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [100]:
df_CanadaPostcodes.rename(columns={'Postcode': 'Postal Code'}, inplace=True)#renaming the first column in the first dataframe to match the second dataframe
df_CanadaPostcodes.head()

,Postal Code,Bourough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The two dataframes are now merged in order to create a new dataframe. The new dataframe displays the postal code of each neighborhood along with the borough name and neighborhood name, and the latitude and the longitude coordinates of each neighborhood.

In [101]:
final_df=pd.merge(df_CanadaPostcodes, geo_data, on='Postal Code')
final_df

,Postal Code,Bourough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
